In [91]:
import copy

# Note: the initialization code for the sudoku board along with the beautiful ASCII printing was taken from:
# https://stackoverflow.com/questions/45471152/how-to-create-a-sudoku-puzzle-in-python
base  = 3

side  = base*base

def create_board(empty_prop=3/4):
    # pattern for a baseline valid solution
    def pattern(r,c): return (base*(r%base)+r//base+c)%side

    # randomize rows, columns and numbers (of valid base pattern)
    from random import sample
    def shuffle(s): return sample(s,len(s)) 
    rBase = range(base) 
    rows  = [ g*base + r for g in shuffle(rBase) for r in shuffle(rBase) ] 
    cols  = [ g*base + c for g in shuffle(rBase) for c in shuffle(rBase) ]
    nums  = shuffle(range(1,base*base+1))
    
    board = [ [nums[pattern(r,c)] for c in cols] for r in rows ]
    answer = copy.deepcopy(board)
    squares = side*side
    empties = int(squares * empty_prop)
    for p in sample(range(squares),empties):
        board[p//side][p%side] = 0
    return answer, board

In [92]:
def print_board(board):
    base = 3
    def expandLine(line):
        return line[0]+line[5:9].join([line[1:5]*(base-1)]*base)+line[9:13]

    line0  = expandLine("╔═══╤═══╦═══╗")
    line1  = expandLine("║ . │ . ║ . ║")
    line2  = expandLine("╟───┼───╫───╢")
    line3  = expandLine("╠═══╪═══╬═══╣")
    line4  = expandLine("╚═══╧═══╩═══╝")

    symbol = " 1234567890ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    nums   = [ [""]+[symbol[n] for n in row] for row in board ]
    print(line0)
    for r in range(1,side+1):
        print( "".join(n+s for n,s in zip(nums[r-1],line1.split("."))) )
        print([line2,line3,line4][(r%side==0)+(r%base==0)])

In [96]:
answer, board = create_board(1/2)
print_board(answer)
print_board(board)

╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 8 │ 7 │ 2 ║ 4 │ 9 │ 1 ║ 6 │ 3 │ 5 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 9 │ 4 │ 1 ║ 3 │ 6 │ 5 ║ 8 │ 7 │ 2 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 6 │ 3 │ 5 ║ 7 │ 8 │ 2 ║ 9 │ 4 │ 1 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 2 │ 8 │ 4 ║ 9 │ 1 │ 3 ║ 5 │ 6 │ 7 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 1 │ 9 │ 3 ║ 6 │ 5 │ 7 ║ 2 │ 8 │ 4 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 5 │ 6 │ 7 ║ 8 │ 2 │ 4 ║ 1 │ 9 │ 3 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 4 │ 2 │ 9 ║ 1 │ 3 │ 6 ║ 7 │ 5 │ 8 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 7 │ 5 │ 8 ║ 2 │ 4 │ 9 ║ 3 │ 1 │ 6 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 3 │ 1 │ 6 ║ 5 │ 7 │ 8 ║ 4 │ 2 │ 9 ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝
╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 8 │ 7 │   ║   │   │   ║ 6 │ 3 │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │ 4 │ 1 ║ 3 │ 6 │   ║   │ 7 │ 2 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 6 │ 3 │   ║ 7 │ 8 │   ║   │   │ 1 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 2 │   │   

In [97]:
rows = len(board)
cols = len(board[0])

def get_possibilities(board):
    missing = set()
    possibilities = [[set() for _ in range(rows)] for _ in range(cols)]
    for row in range(rows):
        for col in range(cols):
            if board[row][col] == 0:
                possibilities[row][col] = set(range(1, 10))
                missing.add((row, col))

    for row in range(rows):
        cur_row = set(board[row])
        for col in range(cols):
            possibilities[row][col] -= cur_row

    for col in range(cols):
        cur_col = set([board[row][col] for row in range(rows)])
        for row in range(cols):
            possibilities[row][col] -= cur_col

    for square in range(9):
        start_row = square // 3
        start_col = square - start_row * 3
        
        start_row *= 3
        start_col *= 3
        
        cur_square = set([board[row][col]
            for row in range(start_row, start_row + 3)
            for col in range(start_col, start_col + 3)
        ])

        for row in range(start_row, start_row + 3):
            for col in range(start_col, start_col + 3):
                possibilities[row][col] -= cur_square
    return missing, possibilities

In [95]:
termination = 25

print_board(board)

for _ in range(termination):
    missing, possibilities = get_possibilities(board)  
    removed = set()
    for m in missing:
        row, col = m
        if len(possibilities[row][col]) == 1:
            board[row][col] = list(possibilities[row][col])[0]

print_board(board)

╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 6 │ 5 │   ║ 2 │   │ 8 ║ 1 │ 9 │ 7 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 8 │ 2 │ 3 ║ 9 │ 7 │ 1 ║   │ 5 │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 1 │ 9 │ 7 ║ 5 │ 4 │   ║   │ 2 │ 3 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 4 │   │   ║ 6 │ 2 │ 3 ║   │ 8 │ 9 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 7 │ 8 │ 9 ║ 1 │   │ 4 ║ 3 │ 6 │ 2 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 3 │   │ 2 ║   │   │   ║ 4 │ 1 │ 5 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║ 3 │ 8 │ 9 ║ 5 │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 5 │ 7 │ 1 ║ 4 │   │   ║ 9 │   │ 8 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 9 │   │   ║   │ 1 │ 5 ║ 2 │   │   ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝
╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 6 │ 5 │ 4 ║ 2 │ 3 │ 8 ║ 1 │ 9 │ 7 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 8 │ 2 │ 3 ║ 9 │ 7 │ 1 ║ 6 │ 5 │ 4 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 1 │ 9 │ 7 ║ 5 │ 4 │ 6 ║ 8 │ 2 │ 3 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 4 │ 1 │ 5 